<a href="https://colab.research.google.com/github/NapsterZ4/saturdays_second_edition/blob/master/cataratas_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Montaje desde google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Importando librerías

In [26]:
import tensorflow as tf
import os
import numpy as np
from sklearn.model_selection import train_test_split

from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Cargando datasets

## Etiquetado y extrayendo información

In [17]:
files = []
labels = []

for dirname, _, filenames in os.walk('/content/drive/My Drive/Colab Notebooks/cataratas_images'):
  for filename in filenames:
    if filename.endswith('.png'):
      files.append(os.path.join(dirname, filename))
      l = np.zeros(4)

      if filename.startswith('NL'):
        l[0] = 1
      
      elif filename.startswith('ca'):
        l[1] = 1

      elif filename.startswith('Gl'):
        l[2] = 1

      elif filename.startswith('Re'):
        l[3] = 1
      
      labels.append(1)

print(len(labels), len(files))

601 601


## Train Test

In [21]:
combined = list(zip(files, labels))
np.random.shuffle(combined)
files[:], labels[:] = zip(*combined)
print("Files:{}, labels:{}".format(files, labels))

Files:['/content/drive/My Drive/Colab Notebooks/cataratas_images/2_glaucoma/Glaucoma_094.png', '/content/drive/My Drive/Colab Notebooks/cataratas_images/2_cataract/cataract_075.png', '/content/drive/My Drive/Colab Notebooks/cataratas_images/1_normal/NL_295.png', '/content/drive/My Drive/Colab Notebooks/cataratas_images/2_cataract/cataract_029.png', '/content/drive/My Drive/Colab Notebooks/cataratas_images/2_glaucoma/Glaucoma_009.png', '/content/drive/My Drive/Colab Notebooks/cataratas_images/1_normal/NL_102.png', '/content/drive/My Drive/Colab Notebooks/cataratas_images/2_cataract/cataract_031.png', '/content/drive/My Drive/Colab Notebooks/cataratas_images/2_glaucoma/Glaucoma_075.png', '/content/drive/My Drive/Colab Notebooks/cataratas_images/1_normal/NL_131.png', '/content/drive/My Drive/Colab Notebooks/cataratas_images/1_normal/NL_223.png', '/content/drive/My Drive/Colab Notebooks/cataratas_images/2_glaucoma/Glaucoma_079.png', '/content/drive/My Drive/Colab Notebooks/cataratas_images

In [ ]:
import re

images = []
directories = []
dircount = []
prevRoot=''
cant=0
 
imgpath = "/content/drive/My Drive/Colab Notebooks/cataratas_images/"

for root, dirnames, filenames in os.walk(imgpath):
    for filename in filenames:
        if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
            cant=cant+1
            filepath = os.path.join(root, filename)
            image = plt.imread(filepath)
            images.append(image)
            b = "Leyendo..." + str(cant)
            print (b, end="\r")
            if prevRoot !=root:
                print(root, cant)
                prevRoot=root
                directories.append(root)
                dircount.append(cant)
                cant=0
dircount.append(cant)
 
dircount = dircount[1:]
dircount[0]=dircount[0]+1
print('Directorios leidos:',len(directories))
print("Imagenes en cada directorio", dircount)
print('suma Total de imagenes en subdirs:',sum(dircount))

/content/drive/My Drive/Colab Notebooks/cataratas_images/1_normal 1
